model is taken from https://coderzcolumn.com/tutorials/artificial-intelligence/eli5-explain-keras-image-classifier-predictions-using-grad-cam

In [2]:
import tensorflow

from tensorflow import keras

print("Tensorflow Version : {}".format(tensorflow.__version__))
print("Keras Version : {}".format(keras.__version__))

Tensorflow Version : 2.12.0
Keras Version : 2.12.0


## Load dataset

In [3]:
import numpy as np

(X_train, Y_train), (X_test, Y_test) = keras.datasets.fashion_mnist.load_data()
#(X_train, Y_train), (X_test, Y_test) = keras.datasets.cifar10.load_data()

X_train, X_test = X_train.reshape(-1,28,28,1), X_test.reshape(-1,28,28,1)

X_train, X_test = X_train/255.0, X_test/255.0

classes =  np.unique(Y_train)
class_labels = ["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]
#class_labels = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]
mapping = dict(zip(classes, class_labels))

X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

4422102/4422102 [==============================] - 0s 0us/step


((60000, 28, 28, 1), (10000, 28, 28, 1), (60000,), (10000,))

## Define and train CNN

In [4]:
from keras.models import Sequential
from keras import layers

model = Sequential([
    layers.Input(shape=X_train.shape[1:]),
    layers.Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"),
    layers.Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu"),
    layers.Conv2D(filters=8, kernel_size=(3,3), padding="same", activation="relu"),

    layers.Flatten(),
    layers.Dense(len(classes), activation="softmax")
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 16)        4624      
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 8)         1160      
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 10)                62730     
                                                                 
Total params: 68,834
Trainable params: 68,834
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.compile("adam", "sparse_categorical_crossentropy", ["accuracy"])

adapt the number of epochs when training the model, 8 epochs is better for accuracy, but less epochs run faster

In [6]:
model.fit(X_train, Y_train, batch_size=256, epochs=2, validation_data=(X_test, Y_test))

Epoch 1/2
235/235 [==============================] - 195s 816ms/step - loss: 0.5431 - accuracy: 0.8098 - val_loss: 0.3865 - val_accuracy: 0.8658
Epoch 2/2
235/235 [==============================] - 170s 726ms/step - loss: 0.3401 - accuracy: 0.8796 - val_loss: 0.3461 - val_accuracy: 0.8792


## Evaluate network performance

In [7]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

Y_test_preds = model.predict(X_test)
Y_test_preds = np.argmax(Y_test_preds, axis=1)

print("Test Accuracy : {}".format(accuracy_score(Y_test, Y_test_preds)))
print("\nConfusion Matrix : ")
print(confusion_matrix(Y_test, Y_test_preds))
print("\nClassification Report :")
print(classification_report(Y_test, Y_test_preds, target_names=class_labels))

313/313 [==============================] - 7s 23ms/step
Test Accuracy : 0.8792

Confusion Matrix : 
[[794   0  32  41   7   3 110   0  13   0]
 [  0 967   3  21   5   0   3   0   1   0]
 [ 11   1 849   7  94   0  37   0   1   0]
 [ 12   4  24 881  47   0  28   0   4   0]
 [  1   1  65  16 887   0  28   0   2   0]
 [  0   0   0   1   0 956   0  29   1  13]
 [ 90   1 131  33 152   1 580   0  12   0]
 [  0   0   0   0   0   8   0 956   1  35]
 [  2   1   8   5   6   2   4   5 967   0]
 [  0   0   0   0   0   3   1  40   1 955]]

Classification Report :
              precision    recall  f1-score   support

 T-shirt/top       0.87      0.79      0.83      1000
     Trouser       0.99      0.97      0.98      1000
    Pullover       0.76      0.85      0.80      1000
       Dress       0.88      0.88      0.88      1000
        Coat       0.74      0.89      0.81      1000
      Sandal       0.98      0.96      0.97      1000
       Shirt       0.73      0.58      0.65      1000
     Sneake

## Grad-CAM

ELI5: https://eli5.readthedocs.io/en/latest/autodocs/keras.html

eli5 requires tensorflow v1 which is not longer supported by google colab
-> cannot be used anymore